In [ ]:
import os

def load_md_files(md_folder_path):
    """
    Lädt alle .md-Dateien aus einem angegebenen Ordner und gibt sie als Liste von Dokumenten zurück.

    :param md_folder_path: Der Pfad zum Ordner, der die .md-Dateien enthält.
    :return: Eine Liste mit dem Inhalt aller .md-Dateien.
    """
    documents = []

    # Durchlaufe alle Dateien im angegebenen Ordner
    for filename in os.listdir(md_folder_path):
        # Überprüfen, ob die Datei eine .md-Datei ist
        if filename.endswith('.md'):
            # Vollständigen Pfad zur Datei erstellen
            file_path = os.path.join(md_folder_path, filename)
            # Datei öffnen und Inhalt lesen
            with open(file_path, 'r', encoding='utf-8') as file:
                # Inhalt zur Dokumentenliste hinzufügen
                documents.append(file.read())

    return documents

# Beispiel: Die Funktion aufrufen
md_folder_path = os.path.join('data', 'md')
documents = load_md_files(md_folder_path)


for i, doc in enumerate(documents):
    print(f"Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

In [ ]:
from transformers import pipeline
import torch

# Verwende ein kleineres Modell wie 't5-small'
device = 0 if torch.cuda.is_available() else -1
corrector = pipeline('text2text-generation', model='t5-small', tokenizer='t5-small', device=device)

# Funktion, um einen langen Text in Abschnitte zu teilen
def split_text(text, max_length=512):
    # Teile den Text in Abschnitte von maximaler Länge (hier 512 Tokens)
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# Funktion, um den Text zu korrigieren und Abschnitte wieder zusammenzusetzen
def correct_long_text(doc, max_length=512):
    # Teile den Text in kürzere Abschnitte
    sections = split_text(doc, max_length)
    corrected_sections = []
    
    # Korrigiere jeden Abschnitt
    for section in sections:
        corrected_section = corrector(f"correct: {section}", clean_up_tokenization_spaces=True)
        corrected_sections.append(corrected_section[0]['generated_text'])
    
    # Setze die korrigierten Abschnitte wieder zu einem Text zusammen
    return ' '.join(corrected_sections)


corrected_documents = []

# Korrigiere jeden langen Text in der Liste
for doc in documents:
    corrected_text = correct_long_text(doc)
    corrected_documents.append(corrected_text)

# Ausgabe der korrigierten Texte
for i, (original, corrected) in enumerate(zip(documents, corrected_documents)):
    print(f"Original Dokument {i+1}: {original}\nKorrigiertes Dokument {i+1}: {corrected}\n")

In [ ]:
import os
import language_tool_python

# Umgebungsvariable setzen, um die Warnung zu unterdrücken
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Lade das LanguageTool-Modul für Deutsch
tool = language_tool_python.LanguageTool('de-DE')

# Funktion zur Korrektur eines langen Textes mit LanguageTool
def correct_text_with_language_tool(text):
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text


corrected_documents = []

# Korrigiere jeden Text in der Liste
for doc in documents:
    corrected_text = correct_text_with_language_tool(doc)
    corrected_documents.append(corrected_text)

# Ausgabe der Original- und korrigierten Texte
for i, (original, corrected) in enumerate(zip(documents, corrected_documents)):
    print(f"Original Dokument {i+1}:\n{original}\n")
    print(f"Korrigiertes Dokument {i+1}:\n{corrected}\n")
    print("="*80 + "\n")

In [ ]:
from IPython.display import display, HTML

def print_corrections(original, corrected):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Originaler Text:</h3>
        <p style="white-space: pre-wrap;">{original}</p>
        <h3>Korrigierter Text:</h3>
        <p style="white-space: pre-wrap; color: green;">{corrected}</p>
    </div>
    """))
    
# Aufruf der Funktion für jedes Dokument
for original, corrected in zip(documents, corrected_documents):
    print_corrections(original, corrected)

In [ ]:
import re
import language_tool_python

# Lade das LanguageTool-Modell für Deutsch
tool = language_tool_python.LanguageTool('de')

# Funktion zur Rechtschreib- und Grammatikprüfung mit LanguageTool
def correct_with_languagetool(text):
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

# Korrigieren von gesplitteten Wörtern
def correct_word_splitting(text):
    return re.sub(r'\b(\w+)\s*-\s*(\w+)\b', r'\1\2', text)

# Hauptfunktion zur Bereinigung der Dokumente
def clean_documents(documents):
    cleaned_documents = []
    
    for doc in documents:
        # Entfernen von problematischen Steuerzeichen (z.B. Steuerzeichen, nicht druckbare Zeichen)
        doc = re.sub(r'[\x00-\x1F\x7F]', ' ', doc)  # Entfernt Steuerzeichen
        
        # Sicherstellen, dass keine doppelten Leerzeichen durch das Entfernen von Zeichen entstehen
        doc = re.sub(r'\s+', ' ', doc).strip()
        
        # Korrigieren von gesplitteten Wörtern
        doc = correct_word_splitting(doc)
        
        # Textkorrektur mit LanguageTool
        doc = correct_with_languagetool(doc)
        
        # Bereinigtes Dokument zur Liste hinzufügen
        cleaned_documents.append(doc)
    
    return cleaned_documents

cleaned_documents = clean_documents(documents)

    
for i, doc in enumerate(cleaned_documents):
    print(f"cleaned_documents {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

In [ ]:
import re

def clean_text(documents):
    # Erlaubte Zeichen: Buchstaben (inkl. ä, ö, ü, ß), Ziffern, Bindestrich, Punkt, Komma, Ausrufezeichen und Fragezeichen
    pattern = r'[^a-zA-ZäöüÄÖÜß0-9\s.,!?-]'
    
    # Entfernt alle unerwünschten Zeichen aus jedem Dokument
    cleaned_documents = [re.sub(pattern, '', doc) for doc in documents]
    return cleaned_documents


# Bereinigte Dokumente
cleaned_documents_2 = clean_text(cleaned_documents)

# Ausgabe der bereinigten Dokumente
for doc in cleaned_documents_2:
    print(doc)

In [ ]:
import spacy

def chunk_documents(documents, chunk_size=8, overlap=2):
    """
    Segmentiert eine Liste von Texten in Sätze und erstellt Chunks mit Overlap.
    
    :param documents: Liste von Texten (Strings), die segmentiert werden sollen
    :param chunk_size: Anzahl der Sätze pro Chunk (z.B. 8 Sätze pro Chunk)
    :param overlap: Anzahl der Sätze, die zwischen aufeinanderfolgenden Chunks überlappen sollen (z.B. 2 Sätze)
    :return: Liste von Chunks, wobei jeder Chunk als separates Dokument gespeichert wird
    """
    # Lade das deutsche Modell
    nlp = spacy.load('de_core_news_sm')
    
    # Liste zum Speichern der Chunks als separate Dokumente
    chunked_documents = []
    
    # Verarbeitung und Segmentierung jedes Dokuments in Sätze
    for text in documents:
        doc = nlp(text)
        sentences = list(doc.sents)
        
        # Chunks erstellen und als eigene Dokumente speichern
        for i in range(0, len(sentences), chunk_size - overlap):
            chunk = sentences[i:i + chunk_size]
            chunked_documents.append(" ".join([sent.text for sent in chunk]))
    
    return chunked_documents

chunked_documents = chunk_documents(cleaned_documents_2, chunk_size=8, overlap=2)

   # Ausgabe aller Chunks
print("Alle Chunks:")
for i, chunk in enumerate(chunked_documents, 1):
    print(f"Chunk {i}: {chunk}\n")

In [ ]:
import spacy
from collections import Counter

def extract_keywords_from_chunk(chunk, top_n=5):
    """
    Extrahiert die häufigsten Substantive und Adjektive aus einem Textchunk.
    
    :param chunk: Ein Textchunk als String
    :param top_n: Anzahl der Top-Keywords, die extrahiert werden sollen
    :return: Liste der Top-Keywords
    """
    nlp = spacy.load('de_core_news_sm')
    doc = nlp(chunk)
    
    # Sammeln von Substantiven und Adjektiven
    keywords = [token.text for token in doc if token.pos_ in ['NOUN', 'ADJ']]
    
    # Häufigkeit der Keywords zählen
    keyword_freq = Counter(keywords)
    
    # Die häufigsten Keywords extrahieren
    most_common_keywords = [keyword for keyword, freq in keyword_freq.most_common(top_n)]
    
    return most_common_keywords

def extract_keywords_from_chunks(chunked_documents, top_n=5):
    """
    Extrahiert die relevantesten Keywords aus jeder Chunk in den gechunkten Dokumenten.
    
    :param chunked_documents: Liste von gechunkten Dokumenten
    :param top_n: Anzahl der Top-Keywords pro Chunk
    :return: Liste von Listen, die Keywords für jeden Chunk enthalten
    """
    all_keywords = []
    
    for chunk in chunked_documents:
        keywords = extract_keywords_from_chunk(chunk, top_n)
        all_keywords.append(keywords)
    
    return all_keywords

# Beispiel für die Verwendung
keywords_per_chunk = extract_keywords_from_chunks(chunked_documents, top_n=5)

# Ausgabe der Keywords pro Chunk
print("Keywords pro Chunk:")
for i, keywords in enumerate(keywords_per_chunk, 1):
    print(f"Chunk {i} Keywords: {keywords}")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_wordcloud(keywords_per_chunk):
    """
    Erstellt eine Wordcloud basierend auf den extrahierten Keywords aus den Chunks.
    
    :param keywords_per_chunk: Liste von Listen, die Keywords für jeden Chunk enthalten
    """
    # Alle Keywords zu einem einzigen String zusammenfügen
    all_keywords = ' '.join([' '.join(keywords) for keywords in keywords_per_chunk])
    
    # WordCloud generieren
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_keywords)
    
    # WordCloud anzeigen
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

# Beispiel für die Verwendung
generate_wordcloud(keywords_per_chunk)

In [ ]:
import os
import pickle

def save_processed_documents(processed_documents, filename):
    # Sicherstellen, dass der Ordner "data/pickle" existiert
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Speichern der Datei im angegebenen Pfad
    with open(filename, "wb") as file:
        pickle.dump(processed_documents, file)
    
    print(f"Die Datei '{filename}' wurde erfolgreich gespeichert.")

# Beispielnutzung:
save_processed_documents(chunked_documents, filename="data/pickle/chunked_doc_list.pkl")
save_processed_documents(keywords_per_chunk, filename="data/pickle/chunk_keywords.pkl")